Strategic Cross-Domain Questions for Fashion Retail Leadership

**Catalog/Schema:** Set in the **Configuration** cell below (session variables `catalog_name`, `schema_name`). Run that cell first, then run the query cells.

- The most impactful executive questions bridge customer behavior and inventory operations to drive demand-driven personalization at scale. Here are the critical questions that require both data domains:

In [ ]:
-- =============================================================================
-- Configuration: Set target catalog and schema for all queries
-- Change these defaults to run against a different catalog/schema
-- =============================================================================
DECLARE OR REPLACE VARIABLE catalog_name STRING DEFAULT 'juan_use1_catalog';
DECLARE OR REPLACE VARIABLE schema_name STRING DEFAULT 'retail';

USE CATALOG IDENTIFIER(catalog_name);
USE SCHEMA IDENTIFIER(schema_name);

SELECT current_catalog(), current_schema(),
       catalog_name || '.' || schema_name AS target;

🎯 Key Insights These Queries Enable

- Demand-Supply Alignment: Identifies where high customer affinity doesn't match inventory availability
- Financial Impact: Quantifies lost revenue from stockouts and tied-up capital from overstock
- Personalization at Scale: Shows how to deliver relevant recommendations constrained by actual inventory
- Omnichannel Optimization: Reveals how customer channel behavior should influence inventory distribution
- Predictive Actions: Combines forecast accuracy with customer behavior to preempt issues  


These queries demonstrate the multiplicative effect where customer behavior data × inventory data = exponentially better decision-making than either dataset alone.

Strategic Cross-Domain Questions for Fashion Retail Leadership
The most impactful executive questions bridge customer behavior and inventory operations to drive demand-driven personalization at scale. Here are the critical questions that require both data domains:

# 📊 CEO/President Level Questions

In [0]:
-- "What's our real-time ability to fulfill demand for trending products across channels?"
WITH weekly_sales AS (
    SELECT 
        s.product_key,
        d.calendar_date,
        SUM(s.quantity_sold) as weekly_quantity_sold
    FROM gold_sales_fact s
    JOIN gold_date_dim d ON s.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 14) -- 2 weeks for lag
    GROUP BY s.product_key, d.calendar_date
),
trending_products AS (
    SELECT 
        cpa.product_key,
        p.product_name,
        p.category_level_2,
        AVG(cpa.affinity_score) as avg_affinity,
        SUM(cpa.view_count) as total_views_last_7d,
        ws.weekly_quantity_sold as units_sold_last_7d,
        ws_prev.weekly_quantity_sold as units_sold_prev_7d,
        (ws.weekly_quantity_sold / NULLIF(ws_prev.weekly_quantity_sold, 0)) - 1 as week_over_week_growth
    FROM gold_customer_product_affinity_agg cpa
    JOIN gold_product_dim p ON cpa.product_key = p.product_key
    JOIN (
        SELECT product_key, SUM(weekly_quantity_sold) as weekly_quantity_sold
        FROM weekly_sales
        WHERE calendar_date >= DATE_SUB(CURRENT_DATE, 7)
        GROUP BY product_key
    ) ws ON cpa.product_key = ws.product_key
    LEFT JOIN (
        SELECT product_key, SUM(weekly_quantity_sold) as weekly_quantity_sold
        FROM weekly_sales
        WHERE calendar_date BETWEEN DATE_SUB(CURRENT_DATE, 14) AND DATE_SUB(CURRENT_DATE, 8)
        GROUP BY product_key
    ) ws_prev ON cpa.product_key = ws_prev.product_key
    WHERE cpa.affinity_score IS NOT NULL
    GROUP BY cpa.product_key, p.product_name, p.category_level_2, ws.weekly_quantity_sold, ws_prev.weekly_quantity_sold
    HAVING week_over_week_growth > 0.5  -- 50%+ growth
),
inventory_coverage AS (
    SELECT 
        tp.product_key,
        tp.product_name,
        tp.week_over_week_growth,
        SUM(CASE WHEN l.channel = 'physical' THEN i.quantity_available ELSE 0 END) as store_inventory,
        SUM(CASE WHEN l.channel = 'digital' THEN i.quantity_available ELSE 0 END) as warehouse_inventory,
        AVG(i.days_of_supply) as avg_days_supply,
        tp.units_sold_last_7d * 4 as projected_monthly_demand
    FROM trending_products tp
    JOIN gold_inventory_fact i ON tp.product_key = i.product_key
    JOIN gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM gold_inventory_fact)
    GROUP BY tp.product_key, tp.product_name, tp.week_over_week_growth, tp.units_sold_last_7d
)
SELECT 
    product_name,
    ROUND(week_over_week_growth * 100, 1) as growth_pct,
    store_inventory,
    warehouse_inventory,
    avg_days_supply,
    projected_monthly_demand,
    CASE 
        WHEN avg_days_supply < 7 THEN 'CRITICAL - Stockout Risk'
        WHEN avg_days_supply < 14 THEN 'WARNING - Low Coverage'
        ELSE 'OK'
    END as inventory_status,
    ROUND((store_inventory + warehouse_inventory) * 100.0 / NULLIF(projected_monthly_demand, 0), 1) as demand_coverage_pct
FROM inventory_coverage
ORDER BY growth_pct DESC;

# CFO

In [0]:
-- "What's the financial impact of misaligned inventory versus customer demand?"
-- Note: affinity_score ranges from 0.03 to 0.15, so we use > 0.08 for high affinity
WITH customer_demand AS (
    -- High affinity products by customer segment
    SELECT 
        c.segment,
        c.geo_region,
        p.product_key,
        p.category_level_2,
        p.base_price,
        COUNT(DISTINCT cpa.customer_key) as interested_customers,
        AVG(cpa.affinity_score) as avg_affinity,
        SUM(cpa.predicted_cltv_impact) as total_cltv_opportunity
    FROM gold_customer_product_affinity_agg cpa
    JOIN gold_customer_dim c ON cpa.customer_key = c.customer_key
    JOIN gold_product_dim p ON cpa.product_key = p.product_key
    WHERE cpa.affinity_score > 0.08  -- High affinity (top ~30%, range: 0.03-0.15)
        AND c.is_current = true
    GROUP BY c.segment, c.geo_region, p.product_key, p.category_level_2, p.base_price
),
inventory_position AS (
    SELECT 
        l.location_key,
        CASE 
            WHEN l.location_name LIKE '%NYC%' OR l.location_name LIKE '%Boston%' THEN 'Northeast'
            WHEN l.location_name LIKE '%LA%' OR l.location_name LIKE '%Seattle%' THEN 'West'
            WHEN l.location_name LIKE '%Miami%' OR l.location_name LIKE '%Atlanta%' THEN 'Southeast'
            ELSE 'Other'
        END as region,
        i.product_key,
        i.quantity_available,
        i.inventory_value_retail,
        i.is_stockout,
        i.is_overstock
    FROM gold_inventory_fact i
    JOIN gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM gold_inventory_fact)
)
SELECT 
    cd.segment,
    cd.geo_region,
    cd.category_level_2,
    SUM(cd.interested_customers) as high_affinity_customers,
    SUM(cd.total_cltv_opportunity) as revenue_opportunity,
    SUM(ip.quantity_available) as available_inventory,
    SUM(ip.inventory_value_retail) as inventory_value,
    SUM(CASE WHEN ip.is_stockout THEN cd.interested_customers ELSE 0 END) as customers_facing_stockout,
    SUM(CASE WHEN ip.is_stockout THEN cd.total_cltv_opportunity ELSE 0 END) as lost_revenue_opportunity,
    SUM(CASE WHEN ip.is_overstock AND cd.avg_affinity < 0.05 THEN ip.inventory_value_retail ELSE 0 END) as overstock_exposure
FROM customer_demand cd
LEFT JOIN inventory_position ip 
    ON cd.product_key = ip.product_key 
    AND cd.geo_region = ip.region
GROUP BY cd.segment, cd.geo_region, cd.category_level_2
ORDER BY lost_revenue_opportunity DESC;

# 🎯 CMO Questions

In [0]:
-- "How do we optimize personalized recommendations based on real-time inventory?"
-- Note: affinity_score ranges from 0.03 to 0.15, so we use > 0.05 (above average)
WITH customer_preferences AS (
    SELECT 
        c.customer_key,
        c.preferred_channel,
        c.nearest_store_id,
        cpa.product_key,
        cpa.affinity_score,
        cpa.predicted_cltv_impact,
        ROW_NUMBER() OVER (PARTITION BY c.customer_key ORDER BY cpa.affinity_score DESC) as preference_rank
    FROM gold_customer_dim c
    JOIN gold_customer_product_affinity_agg cpa ON c.customer_key = cpa.customer_key
    WHERE c.is_current = true 
        AND c.segment = 'regular'  -- Focus on regular customers
        AND cpa.affinity_score > 0.05  -- Above average affinity (range: 0.03-0.15)
),
channel_inventory AS (
    SELECT 
        i.product_key,
        l.channel as fulfillment_channel,  -- 'physical' or 'digital'
        SUM(i.quantity_available) as available_qty,
        MIN(i.days_of_supply) as min_days_supply
    FROM gold_inventory_fact i
    JOIN gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM gold_inventory_fact)
    GROUP BY i.product_key, l.channel
)
SELECT 
    cp.preferred_channel,
    COUNT(DISTINCT cp.customer_key) as customers,
    COUNT(DISTINCT CASE WHEN ci.available_qty > 0 THEN cp.product_key END) as available_recommendations,
    COUNT(DISTINCT cp.product_key) as total_recommendations,
    ROUND(COUNT(DISTINCT CASE WHEN ci.available_qty > 0 THEN cp.product_key END) * 100.0 / 
          NULLIF(COUNT(DISTINCT cp.product_key), 0), 1) as recommendation_fulfillment_rate,
    ROUND(SUM(CASE WHEN ci.available_qty > 0 THEN cp.predicted_cltv_impact ELSE 0 END), 2) as realizable_cltv,
    ROUND(SUM(cp.predicted_cltv_impact) - SUM(CASE WHEN ci.available_qty > 0 THEN cp.predicted_cltv_impact ELSE 0 END), 2) as missed_cltv_opportunity
FROM customer_preferences cp
LEFT JOIN channel_inventory ci ON cp.product_key = ci.product_key
WHERE cp.preference_rank <= 10  -- Top 10 recommendations per customer
GROUP BY cp.preferred_channel
ORDER BY customers DESC;

In [0]:
-- "What's the impact of size/fit issues on inventory efficiency?"
WITH size_performance AS (
    SELECT 
        p.product_key,
        p.product_name,
        p.category_level_2,
        sf.ordered_size,
        COUNT(*) as orders,
        SUM(CASE WHEN sf.is_returned THEN 1 ELSE 0 END) as returns,
        AVG(sf.fit_score) as avg_fit_score
    FROM gold_size_fit_bridge sf
    JOIN gold_product_dim p ON sf.product_key = p.product_key
    GROUP BY p.product_key, p.product_name, p.category_level_2, sf.ordered_size
),
inventory_by_size AS (
    SELECT 
        i.product_key,
        p.size_range,
        SUM(i.quantity_on_hand) as total_inventory,
        SUM(i.inventory_value_retail) as inventory_value,
        AVG(i.days_of_supply) as avg_days_supply
    FROM gold_inventory_fact i
    JOIN gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM gold_inventory_fact)
    GROUP BY i.product_key, p.size_range
)
SELECT 
    sp.category_level_2,
    sp.ordered_size,
    SUM(sp.orders) as total_orders,
    SUM(sp.returns) as total_returns,
    ROUND(SUM(sp.returns) * 100.0 / SUM(sp.orders), 1) as return_rate,
    AVG(sp.avg_fit_score) as avg_fit_score,
    SUM(ibs.total_inventory) as current_inventory,
    SUM(ibs.inventory_value) as tied_up_capital,
    ROUND(SUM(sp.returns) * AVG(ibs.inventory_value) / NULLIF(SUM(ibs.total_inventory), 0), 2) as return_cost_impact
FROM size_performance sp
JOIN inventory_by_size ibs ON sp.product_key = ibs.product_key
GROUP BY sp.category_level_2, sp.ordered_size
HAVING SUM(sp.orders) > 50
ORDER BY return_rate DESC;

# 📦 COO/Supply Chain Questions

In [0]:
-- "How do we optimize inventory allocation based on customer behavior patterns?"
WITH demand_patterns AS (
    SELECT 
        l.location_key,
        l.location_name,
        l.location_type,
        p.category_level_2,
        -- Historical sales velocity
        AVG(s.quantity_sold) as avg_daily_sales,
        STDDEV(s.quantity_sold) as sales_volatility,
        -- Customer engagement
        COUNT(DISTINCT ce.customer_key) as engaged_customers,
        AVG(ce.event_value) as avg_engagement_value,
        -- Forecast accuracy
        AVG(df.forecast_accuracy) as forecast_accuracy,
        AVG(df.mape) as avg_mape
    FROM gold_location_dim l
    LEFT JOIN gold_sales_fact s ON l.location_key = s.location_key
    LEFT JOIN gold_customer_event_fact ce ON l.location_key = ce.location_key
    LEFT JOIN gold_demand_forecast_fact df ON l.location_key = df.location_key
    JOIN gold_product_dim p ON s.product_key = p.product_key
    JOIN gold_date_dim d ON s.date_key = d.date_key
    WHERE d.calendar_date >= DATE_SUB(CURRENT_DATE, 30)
    GROUP BY l.location_key, l.location_name, l.location_type, p.category_level_2
),
current_allocation AS (
    SELECT 
        l.location_key,
        p.category_level_2,
        SUM(i.quantity_on_hand) as current_inventory,
        AVG(i.days_of_supply) as days_of_supply,
        SUM(CASE WHEN i.is_stockout THEN 1 ELSE 0 END) as stockout_skus,
        SUM(CASE WHEN i.is_overstock THEN 1 ELSE 0 END) as overstock_skus
    FROM gold_inventory_fact i
    JOIN gold_location_dim l ON i.location_key = l.location_key
    JOIN gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM gold_inventory_fact)
    GROUP BY l.location_key, p.category_level_2
)
SELECT 
    dp.location_name,
    dp.location_type,
    dp.category_level_2,
    ROUND(dp.avg_daily_sales, 1) as avg_daily_velocity,
    ROUND(dp.sales_volatility / NULLIF(dp.avg_daily_sales, 0), 2) as coefficient_of_variation,
    dp.engaged_customers,
    ca.current_inventory,
    ca.days_of_supply,
    ca.stockout_skus,
    ca.overstock_skus,
    ROUND(dp.forecast_accuracy, 1) as forecast_accuracy_pct,
    -- Recommended inventory level based on demand patterns
    ROUND(dp.avg_daily_sales * 30 * (1 + dp.sales_volatility / NULLIF(dp.avg_daily_sales, 0)), 0) as recommended_inventory,
    ROUND((dp.avg_daily_sales * 30 * (1 + dp.sales_volatility / NULLIF(dp.avg_daily_sales, 0))) - ca.current_inventory, 0) as inventory_gap
FROM demand_patterns dp
JOIN current_allocation ca ON dp.location_key = ca.location_key AND dp.category_level_2 = ca.category_level_2
WHERE dp.avg_daily_sales > 0
ORDER BY ABS(inventory_gap) DESC;

# 🔄 Chief Digital Officer Questions

In [0]:
-- "How does omnichannel behavior affect inventory optimization opportunities?"
WITH omnichannel_customers AS (
    SELECT 
        c.customer_key,
        c.segment,
        COUNT(DISTINCT s.channel_key) as channels_used,
        COUNT(DISTINCT ce.channel_key) as browse_channels,
        CASE 
            WHEN COUNT(DISTINCT s.channel_key) >= 3 THEN 'Omnichannel'
            WHEN COUNT(DISTINCT s.channel_key) = 2 THEN 'Multichannel'
            ELSE 'Single Channel'
        END as channel_behavior
    FROM gold_customer_dim c
    JOIN gold_sales_fact s ON c.customer_key = s.customer_key
    LEFT JOIN gold_customer_event_fact ce ON c.customer_key = ce.customer_key
    WHERE c.is_current = true
    GROUP BY c.customer_key, c.segment
),
customer_value AS (
    SELECT 
        oc.customer_key,
        oc.channel_behavior,
        COUNT(DISTINCT s.transaction_id) as transactions,
        SUM(s.net_sales_amount) as total_revenue,
        SUM(CASE WHEN ca.is_recovered THEN ca.recovery_revenue ELSE 0 END) as recovery_revenue,
        MAX(cpa.affinity_score) as max_affinity_score
    FROM omnichannel_customers oc
    JOIN gold_sales_fact s ON oc.customer_key = s.customer_key
    LEFT JOIN gold_cart_abandonment_fact ca ON oc.customer_key = ca.customer_key
    LEFT JOIN gold_customer_product_affinity_agg cpa ON oc.customer_key = cpa.customer_key
    WHERE s.is_return = false
    GROUP BY oc.customer_key, oc.channel_behavior
),
inventory_fulfillment AS (
    SELECT 
        s.customer_key,
        COUNT(DISTINCT CASE WHEN i.quantity_available > 0 THEN s.product_key END) as fulfilled_products,
        COUNT(DISTINCT s.product_key) as ordered_products,
        COUNT(DISTINCT CASE WHEN i.is_stockout THEN s.product_key END) as stockout_encounters
    FROM gold_sales_fact s
    JOIN gold_inventory_fact i 
        ON s.product_key = i.product_key 
        AND s.location_key = i.location_key
        AND s.date_key = i.date_key
    GROUP BY s.customer_key
)
SELECT 
    cv.channel_behavior,
    COUNT(DISTINCT cv.customer_key) as customer_count,
    ROUND(AVG(cv.total_revenue), 2) as avg_customer_value,
    ROUND(AVG(cv.transactions), 1) as avg_transactions,
    ROUND(AVG(if.fulfilled_products * 100.0 / NULLIF(if.ordered_products, 0)), 1) as fulfillment_rate,
    ROUND(AVG(if.stockout_encounters), 1) as avg_stockout_encounters,
    ROUND(SUM(cv.total_revenue), 2) as total_segment_revenue,
    ROUND(AVG(cv.max_affinity_score), 3) as avg_max_affinity
FROM customer_value cv
JOIN inventory_fulfillment if ON cv.customer_key = if.customer_key
GROUP BY cv.channel_behavior
ORDER BY avg_customer_value DESC;